In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
pwd

'/content/drive/My Drive'

In [20]:
cd Neurosyntactic-Model-for-ABSA-master/ATE/POS+DEP

/content/drive/My Drive/Neurosyntactic-Model-for-ABSA-master/ATE/POS+DEP


In [21]:
pwd

'/content/drive/My Drive/Neurosyntactic-Model-for-ABSA-master/ATE/POS+DEP'

In [22]:
pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 5.9MB/s 
     |████████████████████████████████| 133kB 33.2MB/s 
     |████████████████████████████████| 7.2MB 35.4MB/s 
     |████████████████████████████████| 71kB 5.4MB/s 
ERROR: botocore 1.19.43 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [23]:
pip install transformers

     |████████████████████████████████| 1.5MB 4.2MB/s 
     |████████████████████████████████| 2.9MB 20.7MB/s 
     |████████████████████████████████| 890kB 39.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a0a10ea0e2e1dd0e0a160db79e107b644562d68432f9c700ad28c3b1983dc910
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [24]:
import os
import logging
import argparse
import random
import json
  
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# from pytorch_pretrained_bert.tokenization import BertTokenizer
# from pytorch_pretrained_bert.modeling import PreTrainedBertModel, BertModel
from pytorch_pretrained_bert.optimization import BertAdam
from model import Net
import absa_data_utils as data_utils
import absa_data_utils_test as data_utils_test
from absa_data_utils import ABSATokenizer
# import modelconfig
from torch.autograd import Variable

In [ ]:
# logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
#                     datefmt = '%m/%d/%Y %H:%M:%S',
#                     level = logging.INFO)
# logger = logging.getLogger(__name__)

In [25]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x


In [26]:
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertTokenizer
from transformers import AutoModel, AutoTokenizer

In [27]:
import spacy
nlp = spacy.load('en_core_web_sm')
import pickle

In [29]:
data_dir='../rest'
output_dir='output'

train_batch_size=16
eval_batch_size=8
max_seq_length=400
do_valid=True
learning_rate=3e-5
num_train_epochs=8
warmup_proportion=0.1

In [30]:
def train():
    processor = data_utils.AeProcessor()
    label_list = processor.get_labels()
    tokenizer = ABSATokenizer.from_pretrained('bert-base-uncased')
    train_examples = processor.get_train_examples(data_dir)
    train_features = data_utils.convert_examples_to_features(
        train_examples, label_list, max_seq_length, tokenizer, "ae")
    all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
    
    train_data = TensorDataset(all_input_ids, all_segment_ids, all_input_mask, all_label_ids)

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)
    # pickle.dump(train_dataloader,open("train_32_1.pkl","wb"))
    # train_dataloader=pickle.load(open("train.pkl","rb"))
    num_labels=len(label_list)
    #>>>>> validation
    if do_valid:
        valid_examples = processor.get_dev_examples(data_dir)
        valid_features=data_utils.convert_examples_to_features(
            valid_examples, label_list, max_seq_length, tokenizer, "ae")
        valid_all_input_ids = torch.tensor([f.input_ids for f in valid_features], dtype=torch.long)
        valid_all_segment_ids = torch.tensor([f.segment_ids for f in valid_features], dtype=torch.long)
        valid_all_input_mask = torch.tensor([f.input_mask for f in valid_features], dtype=torch.long)
        valid_all_label_ids = torch.tensor([f.label_id for f in valid_features], dtype=torch.long)
        valid_data = TensorDataset(valid_all_input_ids, valid_all_segment_ids, valid_all_input_mask, valid_all_label_ids)
        valid_sampler = SequentialSampler(valid_data)
        valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=train_batch_size)  

        best_valid_loss=float('inf')
        valid_losses=[]  
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = Net(device=device,num_labels=num_labels).cuda()
    # model = nn.DataParallel(model)

    # Prepare optimizer
    param_optimizer = [(k, v) for k, v in model.named_parameters() if v.requires_grad==True]
    param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    t_total = len(train_dataloader) * num_train_epochs
    optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=learning_rate,
                         warmup=warmup_proportion,
                         t_total=t_total)
    
    global_step = 0
    model.train()
    for q in range(num_train_epochs):
        for step, batch in enumerate(train_dataloader):
            batch = tuple(t.cuda() for t in batch)
            input_ids, segment_ids, input_mask, label_ids = batch
            _,_,_,_,loss=model(input_ids,input_mask, segment_ids,label_ids)
            # loss = model(input_ids, input_mask, segment_ids,label_ids)
            loss.backward()

            lr_this_step = learning_rate * warmup_linear(global_step/t_total, warmup_proportion)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_this_step
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
            #>>>> perform validation at the end of each epoch .
        if do_valid:
            model.eval()
            with torch.no_grad():
                losses=[]
                valid_size=0
                for step, batch in enumerate(valid_dataloader):
                    batch = tuple(t.cuda() for t in batch) # multi-gpu does scattering it-self
                    input_ids, segment_ids, input_mask, label_ids = batch
                    _,_,_,_,loss = model(input_ids, input_mask, segment_ids,label_ids)
                    losses.append(loss.detach().mean().item()*input_ids.size(0) )
                    valid_size+=input_ids.size(0)
                valid_loss=sum(losses)/valid_size
                print("validation loss: %f", valid_loss)
                valid_losses.append(valid_loss)
            # if valid_loss<best_valid_loss:
            if q==2 or q==1 or q==3:
                print("model saved",'',q)
                torch.save(model, os.path.join(output_dir,str(q), "model.pt") )
                best_valid_loss=valid_loss
            model.train()
    if do_valid:
        with open(os.path.join(output_dir, "valid.json"), "w") as fw:
            json.dump({"valid_losses": valid_losses}, fw)
    else:
        torch.save(model, os.path.join(output_dir, "model.pt") )

     

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

In [ ]:
import absa_data_utils_test as data_utils_test

In [ ]:
def test(e):  # Load a trained model that you have fine-tuned (we assume evaluate on cpu)    
    processor = data_utils_test.AeProcessor()
    label_list = processor.get_labels()
    tokenizer = ABSATokenizer.from_pretrained('bert-base-uncased')
    eval_examples = processor.get_test_examples(data_dir)
    eval_features = data_utils_test.convert_examples_to_features(eval_examples, label_list, max_seq_length, tokenizer, "ae")

    print("***** Running evaluation *****")
    print("  Num examples = %d", len(eval_examples))
    print("  Batch size = %d", eval_batch_size)
    all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
    eval_data = TensorDataset(all_input_ids, all_segment_ids, all_input_mask, all_label_ids)
    # Run prediction for full data
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

    model = torch.load(os.path.join(output_dir,str(e), "model.pt") )
    model.cuda()
    model.eval()
    
    full_logits=[]
    full_label_ids=[]
    for step, batch in enumerate(eval_dataloader):
        batch = tuple(t.cuda() for t in batch)
        input_ids, segment_ids, input_mask, label_ids = batch
        
        with torch.no_grad():
            _,logits,_,_,_  = model(input_ids, input_mask, segment_ids,label_ids)

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.cpu().numpy()

        full_logits.extend(logits.tolist() )
        full_label_ids.extend(label_ids.tolist() )

    output_eval_json = os.path.join(output_dir,str(e),"predictions.json") 
    with open(output_eval_json, "w") as fw:
        assert len(full_logits)==len(eval_examples)
        #sort by original order for evaluation
        recs={}
        for qx, ex in enumerate(eval_examples):
            recs[int(ex.guid.split("-")[1]) ]={"sentence": ex.text_a, "idx_map": ex.idx_map, "logit": full_logits[qx][ex.start:]} #skip the [CLS] tag.
        full_logits=[recs[qx]["logit"] for qx in range(len(full_logits))]
        raw_X=[recs[qx]["sentence"] for qx in range(len(eval_examples) ) ]
        idx_map=[recs[qx]["idx_map"] for qx in range(len(eval_examples)) ]
        json.dump({"logits": full_logits, "raw_X": raw_X, "idx_map": idx_map}, fw)


In [ ]:
# for e in range(1,4):
test(2)

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
tokenizer.tokenize('appetizers')